In [ ]:
import geopandas as gpd
import xarray as xr
import rasterio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import  os
import gzip
import shutil
import glob
import rioxarray as rxr

In [2]:
# Set data directory
data_dir = "..\..\datasets\et\lsa_saf"

In [ ]:
# Read AOI vector file



## Evapotranspiration pre-processing

### Read Evapotranspiration file

In [ ]:
''''
Study period: 2004 - 2024
'''
# Path to .tif files
tif_files = glob.glob(data_dir + "/*.tif")

# Extract dates
dates = [
    pd.to_datetime(
        os.path.basename(f).split('.')[2] + '.' + os.path.basename(f).split('.')[3].replace('.tif', ''), 
        format='%Y.%m'
    )
    for f in tif_files
]

print(dates)  